In [13]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm

# Load the non-labeled CSV file
new_data_path = '/Users/rachelwang/Downloads/notes/models/csv/adherence_labeled_with_image_empty.csv'
new_data = pd.read_csv(new_data_path)

# Verify the loaded data
print(new_data.head())

                                    pid  \
0  1f9475bb-f13b-4f68-969b-28f20455b3e7   
1  1f9475bb-f13b-4f68-969b-28f20455b3e7   
2  1f9475bb-f13b-4f68-969b-28f20455b3e7   
3  1f9475bb-f13b-4f68-969b-28f20455b3e7   
4  1f9475bb-f13b-4f68-969b-28f20455b3e7   

                                        task  adherence  \
0                                   Loudness        NaN   
1             Respiration-and-cough-Breath-2        NaN   
2        Respiration-and-cough-FiveBreaths-1        NaN   
3  Respiration-and-cough-ThreeQuickBreaths-2        NaN   
4                   Maximum-phonation-time-1        NaN   

                                                file  \
0  /Users/rachelwang/Downloads/bids_with_sensitiv...   
1  /Users/rachelwang/Downloads/bids_with_sensitiv...   
2  /Users/rachelwang/Downloads/bids_with_sensitiv...   
3  /Users/rachelwang/Downloads/bids_with_sensitiv...   
4  /Users/rachelwang/Downloads/bids_with_sensitiv...   

                                         spectrog

In [14]:
# Define the dataset class
class SpectrogramDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['spectrogram']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Dummy label for compatibility, will not be used
        label = 0
        return image, label

In [15]:
# Define the transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [16]:
# Create a dataset and dataloader for the new data
new_dataset = SpectrogramDataset(new_data, transform=transform)
new_loader = DataLoader(new_dataset, batch_size=32, shuffle=False)

# Load the EfficientNet model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=5)
model.load_state_dict(torch.load('efficientnet_best_model_adherence.pth'))
model = model.to(device)

In [17]:
# Predict labels for the new data
model.eval()
predictions = []

with torch.no_grad():
    for images, _ in new_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Add the predictions to the new data
new_data['adherence'] = [p + 1 for p in predictions]  # Convert back to 1-based labels

# Save the new data with predictions
new_data_with_predictions_path = '/Users/rachelwang/Downloads/notes/models/csv/predicted_adherence_label_empty.csv'
new_data.to_csv(new_data_with_predictions_path, index=False)

print(f"Predictions saved to: {new_data_with_predictions_path}")

Predictions saved to: /Users/rachelwang/Downloads/notes/models/csv/predicted_adherence_label_empty.csv


In [18]:
# Count the occurrences of each adherence label
data = pd.read_csv(new_data_with_predictions_path)
label_counts = new_data['adherence'].value_counts().sort_index()

# Print the counts for each label
print("Adherence label counts:")
print(label_counts)

Adherence label counts:
adherence
1     29
3     35
4     43
5    434
Name: count, dtype: int64
